## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-15-50-14 +0000,nypost,"Russian commander mocks complaining soldier, s...",https://nypost.com/2025/09/13/world-news/russi...
1,2025-09-13-15-41-06 +0000,nyt,Trump Escalates Attacks on Political Opponents...,https://www.nytimes.com/2025/09/13/us/politics...
2,2025-09-13-15-40-20 +0000,nypost,Florida woman fights off 5-foot alligator with...,https://nypost.com/2025/09/13/us-news/florida-...
3,2025-09-13-15-37-55 +0000,nyt,Company That Bought Publishers Clearing House ...,https://www.nytimes.com/2025/09/12/business/pc...
4,2025-09-13-15-34-13 +0000,nypost,Trump ready to hit Moscow with ‘major’ sanctio...,https://nypost.com/2025/09/13/us-news/trump-re...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,kirk,47
20,charlie,42
15,trump,27
112,shooting,17
238,suspect,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
136,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,141
118,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,139
113,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,139
92,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...,135
181,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,135


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
136,141,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...
92,46,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...
177,39,2025-09-12-21-01-59 +0000,bbc,Watch: BBC reports from residence linked to Ki...,https://www.bbc.com/news/videos/cq65y02047lo?a...
21,37,2025-09-13-14-43-23 +0000,nypost,6 more officers fired after woman warned cops ...,https://nypost.com/2025/09/13/us-news/6-more-o...
0,35,2025-09-13-15-50-14 +0000,nypost,"Russian commander mocks complaining soldier, s...",https://nypost.com/2025/09/13/world-news/russi...
104,35,2025-09-13-07-10-14 +0000,bbc,New Nato mission to bolster eastern flank afte...,https://www.bbc.com/news/articles/c0q719qg1wko...
8,32,2025-09-13-15-26-15 +0000,nypost,Zohran Mamdani PAC’s top donors worked for fir...,https://nypost.com/2025/09/13/us-news/top-dono...
7,30,2025-09-13-15-28-33 +0000,nypost,Evergreen High School shooter expressed neo-Na...,https://nypost.com/2025/09/13/us-news/evergree...
242,30,2025-09-12-17-08-33 +0000,nypost,"Roger Marshall, Raphael Warnock push plan to b...",https://nypost.com/2025/09/12/us-news/sens-mar...
14,29,2025-09-13-15-08-00 +0000,wsj,Fed Reserve governor Lisa Cook described one o...,https://www.wsj.com/economy/central-banking/do...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
